In [ ]:
import pandas as pd

: 

# Use TSV to create a modular dataframe

In [3]:
edges = pd.read_csv('data/hetionet-v1.0-edges.tsv', sep='\t', header=0)

In [4]:
nodes = pd.read_csv('data/hetionet-v1.0-nodes.tsv.tsv', sep='\t', header=0)

In [5]:
edges.head()

,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898


In [6]:
set(nodes['kind'])

{'Anatomy',
 'Biological Process',
 'Cellular Component',
 'Compound',
 'Disease',
 'Gene',
 'Molecular Function',
 'Pathway',
 'Pharmacologic Class',
 'Side Effect',
 'Symptom'}

In [7]:
edges.head()

,source,metaedge,target
0,Gene::9021,GpBP,Biological Process::GO:0071357
1,Gene::51676,GpBP,Biological Process::GO:0098780
2,Gene::19,GpBP,Biological Process::GO:0055088
3,Gene::3176,GpBP,Biological Process::GO:0010243
4,Gene::3039,GpBP,Biological Process::GO:0006898


# make map to map IDs with actual name and kind

In [8]:
source_or_target_list_map = nodes['id']

In [9]:
"""" function that makes dictionary from two lists"""
def make_dict(list1, list2):
    return dict(zip(list1, list2))

In [10]:
dict_name = make_dict(source_or_target_list_map, nodes['name'])

In [11]:
dict_kind= make_dict(source_or_target_list_map, nodes['kind'])

In [12]:
final_network_map = edges.copy(deep=True)

In [13]:
final_network_map['source_name'] = final_network_map['source'].map(dict_name)
final_network_map['target_name'] = final_network_map['target'].map(dict_name)


In [14]:
final_network_map['kind_of_source'] = final_network_map['source'].map(dict_kind)

In [15]:
final_network_map['kind_of_target'] = final_network_map['target'].map(dict_kind)

In [28]:
final_network_map.describe()

,source,metaedge,target,source_name,target_name,kind_of_source,kind_of_target
count,2250197,2250197,2250197,2250197,2250197,2250197,2250197
unique,20138,24,44204,20131,44080,5,10
top,Anatomy::UBERON:0000473,GpBP,Gene::7316,testis,UBC,Gene,Gene
freq,23866,559504,2718,23866,2718,1289190,1280427


In [29]:
final_network_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2250197 entries, 0 to 2250196
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   source          object
 1   metaedge        object
 2   target          object
 3   source_name     object
 4   target_name     object
 5   kind_of_source  object
 6   kind_of_target  object
dtypes: object(7)
memory usage: 120.2+ MB


In [25]:
len(list(set(final_network_map[final_network_map['kind_of_source']=='Compound']['source_name'] + final_network_map[final_network_map['kind_of_target']=='Compound']['target_name'])))

6487

In [18]:
list(set(final_network_map['kind_of_source'])) == list(set(final_network_map['kind_of_target']))

False

In [19]:
def find_diff(list1,list2):
    return (list(set(list1) - set(list2)))

In [20]:
list(set(final_network_map['kind_of_source']))

['Gene', 'Compound', 'Anatomy', 'Pharmacologic Class', 'Disease']

In [21]:
list(set(final_network_map['kind_of_target']))

['Cellular Component',
 'Pathway',
 'Gene',
 'Compound',
 'Symptom',
 'Anatomy',
 'Molecular Function',
 'Side Effect',
 'Disease',
 'Biological Process']

In [22]:
find_diff(list(set(final_network_map['kind_of_source'])), list(set(final_network_map['kind_of_target'])))

['Pharmacologic Class']

In [23]:
meta_edge_encoding = pd.read_csv('data/meta_edges_encoding.tsv', sep='\t', header=0)

In [24]:
set(meta_edge_encoding['abbreviation']) == set(final_network_map['metaedge'])

True

## since the meta_edge_ecoding is same as final_network_map metaedges we can encode them

In [25]:
met_edge_map = make_dict(meta_edge_encoding['abbreviation'], meta_edge_encoding['metaedge'])

In [26]:
final_network_map['metaedge'] = final_network_map['metaedge'].map(met_edge_map)

In [32]:
final_network_map.to_csv('data/final_network_map.csv', index=False)

In [27]:
list(set(final_network_map['metaedge']))

['Gene - participates - Molecular Function',
 'Disease - presents - Symptom',
 'Disease - downregulates - Gene',
 'Disease - upregulates - Gene',
 'Compound - causes - Side Effect',
 'Disease - resembles - Disease',
 'Compound - downregulates - Gene',
 'Disease - associates - Gene',
 'Gene > regulates > Gene',
 'Compound - upregulates - Gene',
 'Anatomy - expresses - Gene',
 'Pharmacologic Class - includes - Compound',
 'Disease - localizes - Anatomy',
 'Gene - participates - Pathway',
 'Anatomy - downregulates - Gene',
 'Gene - participates - Cellular Component',
 'Gene - participates - Biological Process',
 'Compound - binds - Gene',
 'Compound - resembles - Compound',
 'Compound - treats - Disease',
 'Compound - palliates - Disease',
 'Anatomy - upregulates - Gene',
 'Gene - interacts - Gene',
 'Gene - covaries - Gene']

# now we can use this csv to make a streamlit app

In [46]:
df_gene = pd.concat([final_network_map[final_network_map['kind_of_source']=='Gene'],final_network_map[final_network_map['kind_of_target']=='Gene']]).reset_index(drop=True)  

In [60]:
df_disease = (pd.concat([final_network_map[final_network_map['kind_of_source']=='Disease'],final_network_map[final_network_map['kind_of_target']=='Disease']]).reset_index(drop=True)).drop_duplicates(keep='first')

In [68]:
len((list(set(df_disease[df_disease['kind_of_source']=='Disease']['source_name']))+list(set(df_disease[df_disease['kind_of_target']=='Disease']['source_name']))))

799

In [75]:
set(df_disease['kind_of_target'])

{'Anatomy', 'Disease', 'Gene', 'Symptom'}

In [76]:
df_disease.to_csv('df_disease.csv', index=False)

In [50]:
df_gene_disease = (pd.concat([df_gene,df_disease]).reset_index(drop=True)).drop_duplicates(keep='first').reset_index(drop=True)

In [57]:
df_target_disease = pd.concat([df_gene_disease[df_gene_disease['kind_of_target']=='Disease'],df_gene_disease[df_gene_disease['kind_of_source']=='Disease']])

In [58]:
df_target_disease = df_target_disease.reset_index(drop=True)

In [59]:
df_target_disease

,source,metaedge,target,source_name,target_name,kind_of_source,kind_of_target
0,Disease::DOID:2994,Disease - resembles - Disease,Disease::DOID:11934,germ cell cancer,head and neck cancer,Disease,Disease
1,Disease::DOID:219,Disease - resembles - Disease,Disease::DOID:3571,colon cancer,liver cancer,Disease,Disease
2,Disease::DOID:263,Disease - resembles - Disease,Disease::DOID:4045,kidney cancer,muscle cancer,Disease,Disease
3,Disease::DOID:1964,Disease - resembles - Disease,Disease::DOID:13223,fallopian tube cancer,uterine fibroid,Disease,Disease
4,Disease::DOID:184,Disease - resembles - Disease,Disease::DOID:10283,bone cancer,prostate cancer,Disease,Disease
...,...,...,...,...,...,...,...
37162,Disease::DOID:219,Disease - resembles - Disease,Disease::DOID:8577,colon cancer,ulcerative colitis,Disease,Disease
37163,Disease::DOID:2994,Disease - resembles - Disease,Disease::DOID:13499,germ cell cancer,jejunal cancer,Disease,Disease
37164,Disease::DOID:1793,Disease - resembles - Disease,Disease::DOID:10534,pancreatic cancer,stomach cancer,Disease,Disease
37165,Disease::DOID:219,Disease - resembles - Disease,Disease::DOID:3121,colon cancer,gallbladder cancer,Disease,Disease


In [42]:
gene_disease = df_gene[df_gene['kind_of_target']=='Disease']

In [41]:
gene_disease

,source,metaedge,target,source_name,target_name,kind_of_source,kind_of_target


In [78]:
list(set(list(set(final_network_map['kind_of_source'])) + list(set(final_network_map['kind_of_target']))))

['Cellular Component',
 'Pathway',
 'Gene',
 'Compound',
 'Symptom',
 'Anatomy',
 'Molecular Function',
 'Side Effect',
 'Pharmacologic Class',
 'Disease',
 'Biological Process']

In [30]:
list(set(final_network_map['kind_of_target']))

['Cellular Component',
 'Pathway',
 'Gene',
 'Compound',
 'Symptom',
 'Anatomy',
 'Molecular Function',
 'Side Effect',
 'Disease',
 'Biological Process']

In [31]:
final_network_map.to_csv("data/final_network_map" + '.bz2', \
                                           index=False, \
                                           compression='bz2')

In [32]:
import os

In [51]:
final_network_map_read = !timeit -n5 -r5 -o pd.read_csv("data/final_network_map" + '.gzip', \
                        z                     compression='gzip')


3.5 s ± 10.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [52]:
final_network_map_read

<TimeitResult : 3.5 s ± 10.4 ms per loop (mean ± std. dev. of 5 runs, 5 loops each)>

In [ ]:
final_network_map_read = !timeit -n5 -r5 -o pd.read_csv("data/final_network_map" + '.gzip', \
                        z                     compression='gzip')


# Use JSON to create a dataframe

metagraph = pd.read_jsc